In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()
market_train, news_train = market_train_df, news_train_df

In [ ]:
%%time
# The target is binary
target = market_train.returnsOpenNextMktres10

fcol_market = [c for c in market_train if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]
fcol_news = [c for c in news_train if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]

In [ ]:
fcol_news

In [ ]:
type(target)

In [ ]:
X = market_train[fcol]
target = target

In [ ]:
type(target)
type(X)



In [ ]:
from sklearn.preprocessing import StandardScaler
#df_sc = StandardScaler().fit_transform(X)

In [ ]:
#import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(X)

In [ ]:
df_sc = scaler.fit_transform(X)

In [ ]:
df_scaler = pd.DataFrame(df_sc).head()

In [ ]:
from xgboost import XGBRegressor
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, target, test_size=0.25, random_state=99)

xgb_up = XGBRegressor(n_jobs=-1,n_estimators=200,max_depth=8,eta=0.1)
print('Fitting Up')
xgb_up.fit(X_train,y_train)
print("Accuracy Score: ",accuracy_score(xgb_up.predict(X_test),y_test))

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
x_imputed = my_imputer.fit_transform(X)

In [ ]:
from sklearn.preprocessing import StandardScaler
x_imputed_sc = StandardScaler().fit_transform(x_imputed)

In [ ]:
x_imputed_sc_df = pd.DataFrame(x_imputed_sc)

In [ ]:
x_imputed_sc_df.head()

In [ ]:
import time

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x_imputed_sc_df, target, test_size=0.25, random_state=99)
start = time.time()
xgb_up = XGBRegressor(n_jobs=-1,n_estimators=20,max_depth=8,eta=0.1)
print('Fitting Up')
xgb_up.fit(X_train,y_train)
end = time.time()
print(end-start)


In [ ]:
A=xgb_up.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,A)